In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List

from IPython.display import HTML, clear_output

import json
import numpy as np
import mujoco
from brax.tools import mujoco as braxmjc

from mujoco_html import CartesianPose, render

In [3]:
# Load the the model.
model = mujoco.MjModel.from_xml_path("./ant_scene.xml")
data = mujoco.MjData(model)

with open("./ant_scene.xml", "r") as f:
    xml_string = f.read()

m = braxmjc.MujocoConverter(
    xml_string,
    ignore_unsupported_joints=True,
    add_collision_pairs=True,
)
config = m.config
config.dt = model.opt.timestep

In [4]:
# This will contain Cartesian poses of the bodies in the world frame.
frames: List[CartesianPose] = []

# Apply random controls for a 2 second duration.
while data.time < 2.0:
    data.ctrl = np.random.uniform(*model.actuator_ctrlrange.T)
    mujoco.mj_step(model, data)
    frames.append(CartesianPose.from_mjdata(data))

In [5]:
HTML(render(config, frames))